### Cleaning Data


In [26]:
import duckdb

In [27]:
con = duckdb.connect(database = "jobinfo.db")
temp_view = con.sql("SELECT * FROM jobinfo").df() #running sql directly breaks output on github
temp_view.head()

,id,site,job_url,title,company,location,job_type,date_posted,job_level,job_function,company_industry,description
0,caec8ea1c08a3b7d,indeed,https://uk.indeed.com/viewjob?jk=caec8ea1c08a3b7d,BI Analyst,Quotient Sciences,"Ruddington, ENG, GB",fulltime,2024-10-16,None,None,Pharmaceuticals,Quotient Sciences is a drug development and ma...
1,7b75319b0fa3cabb,indeed,https://uk.indeed.com/viewjob?jk=7b75319b0fa3cabb,Data & Analytics Analyst (SQL and Remediation ...,AMS,"Edinburgh, SCT, GB",fulltime,2024-10-16,None,None,Human Resources And Staffing,**Responsibilities of the role:** **As a** **D...
2,8235d10054b56f3a,indeed,https://uk.indeed.com/viewjob?jk=8235d10054b56f3a,Business Systems Analyst,idverde,"Coventry, ENG, GB",fulltime,2024-10-16,None,None,None,Description\n\n**Business Systems Analyst****L...
3,f977a670fa1c7366,indeed,https://uk.indeed.com/viewjob?jk=f977a670fa1c7366,Senior PMO Analyst,Close Brothers,"London, ENG, GB",None,2024-10-16,None,None,None,**OVERALL SUMMARY**\n\n\nAt Close Brothers we ...
4,2ab2a410123f9ffc,indeed,https://uk.indeed.com/viewjob?jk=2ab2a410123f9ffc,Business Analyst,Bupa,"London, ENG, GB",fulltime,2024-09-16,None,None,None,Business Analyst\n\n\nJob Description:\n\n**Se...


In [28]:
titles = con.sql("SELECT DISTINCT title from jobinfo").df()
titles.head()

,title
0,BI Analyst
1,Data & Analytics Analyst (SQL and Remediation ...
2,Business Systems Analyst
3,Data Analyst (Governance)
4,Business Intelligence Specialist


In [29]:
#Dropping jobs that have no description
con.execute("DELETE FROM jobinfo WHERE description IS NULL")



#Dropping Junior jobs
title_pattern1 = '%Junior%'
title_pattern2 = '%junior%'
title_pattern3 = '%manager%'
title_pattern4 = '%Manager%'
title_pattern5 = '%graduate%'
title_pattern6 = '%Graduate%'
title_pattern7 = '%Trainee%'
title_pattern8 = '%Intern%'
con.execute(" DELETE FROM jobinfo WHERE title LIKE ?",[title_pattern1]).fetchall()
con.execute(" DELETE FROM jobinfo WHERE title LIKE ?",[title_pattern2]).fetchall()
con.execute(" DELETE FROM jobinfo WHERE title LIKE ?",[title_pattern3]).fetchall()
con.execute(" DELETE FROM jobinfo WHERE title LIKE ?",[title_pattern4]).fetchall()
con.execute(" DELETE FROM jobinfo WHERE title LIKE ?",[title_pattern5]).fetchall()
con.execute(" DELETE FROM jobinfo WHERE title LIKE ?",[title_pattern6]).fetchall()
con.execute(" DELETE FROM jobinfo WHERE title LIKE ?",[title_pattern7]).fetchall()
con.execute(" DELETE FROM jobinfo WHERE title LIKE ?",[title_pattern8]).fetchall()


con.sql("SELECT * FROM jobinfo")

┌──────────────────┬──────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────┬─────────────┬──────────────────┬──────────────────────┬──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [30]:
#Dropping columns that arent particularly useful,also attempting to write a for loop and function for this to avoid a situation like previous code block
columns_to_drop = ['job_url_direct','min_amount','max_amount','is_remote','listing_type','emails','company_url','company_url_direct','company_addresses','company_num_employees','company_revenue','company_description','logo_photo_url','banner_photo_url','ceo_name','ceo_photo_url','salary_source','interval','currency']

def column_check(conn,table_name,column_name):
    result = conn.execute(f"""
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = '{table_name}' AND column_name = '{column_name}'
    """).fetchone()
    return result is not None

def mult_column_drop(conn, table_name, columns):
    for column in columns:
        if column_check(conn,table_name,column):
            print(f"Column '{column}' exists, it will be dropped")
            conn.execute(f"""
                          ALTER TABLE {table_name} DROP {column}
                          """)
        else:
            print(f"""Column '{column} does not exist, moving on to next column""")
                   
mult_column_drop(con,'jobinfo',columns_to_drop)


Column 'job_url_direct does not exist, moving on to next column
Column 'min_amount does not exist, moving on to next column
Column 'max_amount does not exist, moving on to next column
Column 'is_remote does not exist, moving on to next column
Column 'listing_type does not exist, moving on to next column
Column 'emails does not exist, moving on to next column
Column 'company_url does not exist, moving on to next column
Column 'company_url_direct does not exist, moving on to next column
Column 'company_addresses does not exist, moving on to next column
Column 'company_num_employees does not exist, moving on to next column
Column 'company_revenue does not exist, moving on to next column
Column 'company_description does not exist, moving on to next column
Column 'logo_photo_url does not exist, moving on to next column
Column 'banner_photo_url does not exist, moving on to next column
Column 'ceo_name does not exist, moving on to next column
Column 'ceo_photo_url does not exist, moving on to

In [31]:
temp_view = con.sql("SELECT * FROM jobinfo").df() #running sql directly breaks output on github
temp_view.head()
#nice!! a lot trimmer , now to clean description

┌─────────┬─────────┬─────────┬─────────┬─────────┬──────────┬──────────┬─────────────┬───────────┬──────────────┬──────────────────┬─────────────┐
│   id    │  site   │ job_url │  title  │ company │ location │ job_type │ date_posted │ job_level │ job_function │ company_industry │ description │
│ varchar │ varchar │ varchar │ varchar │ varchar │ varchar  │ varchar  │    date     │  varchar  │   varchar    │     varchar      │   varchar   │
├─────────┴─────────┴─────────┴─────────┴─────────┴──────────┴──────────┴─────────────┴───────────┴──────────────┴──────────────────┴─────────────┤
│                                                                     0 rows                                                                      │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

,id,site,job_url,title,company,location,job_type,date_posted,job_level,job_function,company_industry,description
0,caec8ea1c08a3b7d,indeed,https://uk.indeed.com/viewjob?jk=caec8ea1c08a3b7d,BI Analyst,Quotient Sciences,"Ruddington, ENG, GB",fulltime,2024-10-16,None,None,Pharmaceuticals,Quotient Sciences is a drug development and ma...
1,7b75319b0fa3cabb,indeed,https://uk.indeed.com/viewjob?jk=7b75319b0fa3cabb,Data & Analytics Analyst (SQL and Remediation ...,AMS,"Edinburgh, SCT, GB",fulltime,2024-10-16,None,None,Human Resources And Staffing,**Responsibilities of the role:** **As a** **D...
2,8235d10054b56f3a,indeed,https://uk.indeed.com/viewjob?jk=8235d10054b56f3a,Business Systems Analyst,idverde,"Coventry, ENG, GB",fulltime,2024-10-16,None,None,None,Description\n\n**Business Systems Analyst****L...
3,f977a670fa1c7366,indeed,https://uk.indeed.com/viewjob?jk=f977a670fa1c7366,Senior PMO Analyst,Close Brothers,"London, ENG, GB",None,2024-10-16,None,None,None,**OVERALL SUMMARY**\n\n\nAt Close Brothers we ...
4,2ab2a410123f9ffc,indeed,https://uk.indeed.com/viewjob?jk=2ab2a410123f9ffc,Business Analyst,Bupa,"London, ENG, GB",fulltime,2024-09-16,None,None,None,Business Analyst\n\n\nJob Description:\n\n**Se...
